# VEGETABLE OIL

- https://bashtage.github.io/linearmodels/iv/introduction.html

Cap 15 Wooldridge - pg 494

In [1]:
# !pip install linearmodels fredapi --quiet

In [40]:
import pandas as pd
import numpy as np
import linearmodels
import tabulate
import requests

pd.set_option('display.max_columns', None)

# Leitura do Dataset

In [41]:
def get_main():
    df = pd.read_csv('https://raw.githubusercontent.com/fkfouri/vegetable_oil_mkt/master/dataset/COMM5__v2.csv', skiprows=1)
    print(df.shape)
    df['const'] = 1
    df.dropna(axis=1, how='all')
    print(df.shape)

    df['date'] = pd.to_datetime(df['date'])
    # df['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d'))
    df.set_index('date', inplace=True)
    return df

df_main = get_main()
df_main.head()


(245, 93)
(245, 94)


,reference,A_exports,A_price,A_production,A_share_PROD,A_share_export,A_ΔP,A_ΔQ,A_Cuzada,A_Elasticidade,A_vol_prod,A_vol_exp,A_PnQo,A_balance,B_exports,B_price,B_production,B_share_PROD,B_share_export,B_ΔP,B_ΔQ,B_Cuzada,B_Elasticidade,B_vol_prod,B_vol_exp,B_PnQo,B_balance,C_exports,C_price,C_production,C_share_PROD,C_share_export,C_ΔP,C_ΔQ,C_Cuzada,C_Elasticidade,C_vol_prod,C_vol_exp,C_PnQo,C_balance,D_exports,D_price,D_production,D_share_PROD,D_share_export,D_ΔP,D_ΔQ,D_Cuzada,D_Elasticidade,D_vol_prod,D_vol_exp,D_PnQo,D_balance,E_exports,E_price,E_production,E_share_PROD,E_share_export,E_ΔP,E_ΔQ,E_Cuzada,E_Elasticidade,E_vol_prod,E_vol_exp,E_PnQo,E_balance,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,FK IDX CALC,Δ IDX/FAO,FK IDX 5 COMM,Δ IDX/FAO.1,FK IDX 7 COMM,Δ IDX/FAO.2,vegetable-oil_price,vegetable-oil_exports,EXPORT\nDIF USDA,EXPORT\nCALC/USDA,vegetable-oil_production,VOL PROD \nCALC/USDA,vegetable-oil_oecd_exports,EXPORT \nDIF OECD,EXPORT \nCALC/OECD,vegetable-oil_oecd_production,VOL PROD\nCALC/OECD,const
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-04-30,NaN,NaN,2064.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0000,546815.0759,NaN,NaN,1682.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,6548079.817,NaN,NaN,2266.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,760390.6818,NaN,NaN,1947.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,2995036.679,NaN,NaN,2275.760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,1.422259e+06,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.227258e+07,NaN,0.000,NaN,229.524,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7367.33,NaN,NaN,18396.90,NaN,1
2022-03-31,NaN,NaN,2441.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0000,646723.6372,NaN,NaN,1776.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,6914719.987,NaN,NaN,2162.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,725538.1420,NaN,NaN,1956.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,3009446.614,NaN,NaN,2361.130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,1.475612e+06,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.277204e+07,NaN,0.000,0.000,238.865,0.961,233.729531,0.940,248.598,NaN,NaN,NaN,NaN,NaN,7359.91,NaN,NaN,18374.57,NaN,1
2022-02-28,NaN,NaN,2442.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0000,647028.2607,NaN,NaN,1522.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,5923989.915,NaN,NaN,1744.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,585248.8680,NaN,NaN,1595.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,2454056.631,NaN,NaN,1499.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,9.368900e+05,NaN,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.054721e+07,NaN,0.000,0.000,197.256,0.978,199.921231,0.991,201.718,NaN,NaN,NaN,NaN,NaN,7352.49,NaN,NaN,18352.24,NaN,1
2022-01-31,NaN,NaN,2195.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0000,581642.8131,NaN,NaN,1344.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,5233008.223,NaN,NaN,1865.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,625902.8944,NaN,NaN,1469.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,2260006.933,NaN,NaN,1411.730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,8.822748e+05,NaN,0.00,0.00,0.00,0.00,-2.000,-2.000,1.00,9.582836e+06,NaN,0.000,0.000,179.220,0.964,181.332226,0.975,185.931,NaN,NaN,NaN,NaN,NaN,7345.07,NaN,NaN,18329.91,NaN,1
2021-12-31,World,283.17,1861.43,725.0,4.15,3.7,-0.097,0.006,-0.02,0.059,1349536.75,527101.1331,493074.1927,39.1,4136.08,1270.29,6299.5,36.04,54.69,-0.054,0.003,0.021,0.049,8002191.855,5254021.063,4943104.883,65.7,459.5,1742.50,2356.42,13.48,6.08,-0.046,-0.013,0.0,0.281,4106061.85,800678.75,584678.4500,19.5,1669.67,1411.21,6340.5,36.27,22.08,-0.022,0.001,0.0,0.066,8947777.005,2356255.001,2170271.635,26.3,1014.0,1361.831,1759.33,10.06,13.41,-0.039,0.006,0.23,0.165,2395910.133,1380896.634,8.510899e+05,57.6,17480.75,7562.42,24801477.59,10318952.58,-0.042,-0.045,1.05,9.042219e+06,1418.79,172.586,0.967,169.109,0.947,169.982419,0.952,178.505,7887.34,324.92,0.958805,19012.5,0.919435,7337.65,-224.77,1.030632,18307.58,0.954837,1


In [42]:
[col for col in df_main.columns.to_list() if 'total' in col]

['prod_total',
 'exp_total',
 'vol_prod_total',
 'vol_expo_total',
 'ΔP_total',
 'ΔQ_total',
 'elasticidade_total',
 'ΣPnQo_total',
 'price_total']

 # Variavel Instrumental

- https://fred.stlouisfed.org/series/PCU325311325311
- https://fertilizerpricing.com/priceindex/
 

In [43]:
def get_fred():
    from fredapi import Fred
    from datetime import datetime, timedelta, date, time

    fred = Fred(api_key='34226d58c6bcee68e056cb4c42b8f401')

    df_temp = fred.get_series('PCU325311325311').to_frame()
    df_temp = df_temp.reset_index()
    df_temp.columns = ['date', 'fred_price_idx']


    df_temp['date'] = df_temp.date - pd.Timedelta(days=1)

    df_temp.set_index('date', inplace=True)
    # df_sun = pd.merge(df_sun, df_temp, how="outer", left_index=True, right_index=True)
    # df_sun.tail()
    return df_temp

df1 = get_fred()
print(df1.shape)
df1

(556, 1)


,fred_price_idx
date,
1975-11-30,88.100
1975-12-31,87.600
1976-01-31,85.000
1976-02-29,84.800
1976-03-31,84.700
...,...
2021-10-31,539.354
2021-11-30,615.654
2021-12-31,643.475


In [44]:
def get_green_marks():
    req = requests.get('https://fertilizerpricing.com/wp-content/themes/greenmarkets/fcharts/fchart_lib/json/data_open.php', headers=None)
    temp = req.json()
    df = pd.DataFrame(temp[0]['data'],  columns = ['date', 'bloomberg_price_index'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    df['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d'))

    df = df.groupby(pd.PeriodIndex(df['date'], freq="M"))['bloomberg_price_index'].mean().reset_index()
    df['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d'))
    df.set_index('date', inplace=True)
    return df

df2 = get_green_marks()
print(df2.shape)
df2

(245, 1)


,bloomberg_price_index
date,
2002-01-31,99.3250
2002-02-28,98.6600
2002-03-31,98.7175
2002-04-30,98.0920
2002-05-31,101.5725
...,...
2022-01-31,886.6000
2022-02-28,883.6575
2022-03-31,1160.1925


# Merging

In [45]:
df_ref = df_main.copy()
print(df_ref.shape)
df_ref  = pd.merge(df_ref, df1, how="left", left_index=True, right_index=True)
print(df_ref.shape)
df_ref  = pd.merge(df_ref, df2, how="left", left_index=True, right_index=True)
print(df_ref.shape)
df_ref

(245, 93)
(245, 94)
(245, 95)


,reference,A_exports,A_price,A_production,A_share_PROD,A_share_export,A_ΔP,A_ΔQ,A_Cuzada,A_Elasticidade,A_vol_prod,A_vol_exp,A_PnQo,A_balance,B_exports,B_price,B_production,B_share_PROD,B_share_export,B_ΔP,B_ΔQ,B_Cuzada,B_Elasticidade,B_vol_prod,B_vol_exp,B_PnQo,B_balance,C_exports,C_price,C_production,C_share_PROD,C_share_export,C_ΔP,C_ΔQ,C_Cuzada,C_Elasticidade,C_vol_prod,C_vol_exp,C_PnQo,C_balance,D_exports,D_price,D_production,D_share_PROD,D_share_export,D_ΔP,D_ΔQ,D_Cuzada,D_Elasticidade,D_vol_prod,D_vol_exp,D_PnQo,D_balance,E_exports,E_price,E_production,E_share_PROD,E_share_export,E_ΔP,E_ΔQ,E_Cuzada,E_Elasticidade,E_vol_prod,E_vol_exp,E_PnQo,E_balance,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,FK IDX CALC,Δ IDX/FAO,FK IDX 5 COMM,Δ IDX/FAO.1,FK IDX 7 COMM,Δ IDX/FAO.2,vegetable-oil_price,vegetable-oil_exports,EXPORT\nDIF USDA,EXPORT\nCALC/USDA,vegetable-oil_production,VOL PROD \nCALC/USDA,vegetable-oil_oecd_exports,EXPORT \nDIF OECD,EXPORT \nCALC/OECD,vegetable-oil_oecd_production,VOL PROD\nCALC/OECD,const,fred_price_idx,bloomberg_price_index
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-04-30,NaN,NaN,2064.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,546815.0759,NaN,NaN,1682.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,6548079.817,NaN,NaN,2266.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,760390.6818,NaN,NaN,1947.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000000e+00,2.995037e+06,NaN,NaN,2275.760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,1.422259e+06,NaN,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.227258e+07,NaN,0.000,NaN,229.524,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7367.33,NaN,NaN,18396.90,NaN,1,NaN,1154.830000
2022-03-31,NaN,NaN,2441.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,646723.6372,NaN,NaN,1776.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,6914719.987,NaN,NaN,2162.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,725538.1420,NaN,NaN,1956.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000000e+00,3.009447e+06,NaN,NaN,2361.130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,1.475612e+06,NaN,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.277204e+07,NaN,0.000,0.000,238.865,0.961,233.729531,0.940,248.598,NaN,NaN,NaN,NaN,NaN,7359.91,NaN,NaN,18374.57,NaN,1,NaN,1160.192500
2022-02-28,NaN,NaN,2442.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,647028.2607,NaN,NaN,1522.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,5923989.915,NaN,NaN,1744.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,585248.8680,NaN,NaN,1595.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000000e+00,2.454057e+06,NaN,NaN,1499.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,9.368900e+05,NaN,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.054721e+07,NaN,0.000,0.000,197.256,0.978,199.921231,0.991,201.718,NaN,NaN,NaN,NaN,NaN,7352.49,NaN,NaN,18352.24,NaN,1,674.724,883.657500
2022-01-31,NaN,NaN,2195.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,581642.8131,NaN,NaN,1344.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,5233008.223,NaN,NaN,1865.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0000,625902.8944,NaN,NaN,1469.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000000e+00,2.260007e+06,NaN,NaN,1411.730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,8.822748e+05,NaN,0.00,0.00,0.000000e+00,0.000000e+00,-2.000,-2.000,1.000,9.582836e+06,NaN,0.000,0.000,179.220,0.964,181.332226,0.975,185.931,NaN,NaN,NaN,NaN,NaN,7345.07,NaN,NaN,18329.91,NaN,1,677.806,886.600000
2021-12-31,World,283.17,1861.43,725.00,4.15,3.7,-0.097,0.006,-0.020,0.059,1.349537e+06,527101.1331,493074.1927,39.1,4136.08,1270.29,6299.50,36.04,54.69,-0.054,0.003,0.021,0.049,8.002192e+06,5.254021e+06,4943104.883,65.7,459.50,1742.50,2356.42,13.48,6.08,-0.046,-0.013,0.0,0.281,4.106062e+06,800678.7500,584678.4500,19.5,1669.67,1411.21,6340.50,36.27,22.08,-0.022,0.001,0.0,0.066,8947777.005,2.356255e+06,2.170272e+06,26.3,1014.00,1361.831,1759.33,1

In [47]:
df_ref.to_excel(r'E:\Projetos\vegetable_oil_mkt\dataset\COMM5__com_fertilizer.xlsx', index=True)

In [24]:
df_ref = df_main[[col for col in df_main.columns.to_list() if 'total' in col] + ['vegetable-oil_price', 'const'] ]
print(df_ref.shape)
df_ref

(245, 11)


,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,vegetable-oil_price,const
date,,,,,,,,,,,
2022-04-30,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.227258e+07,NaN,NaN,1
2022-03-31,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.277204e+07,NaN,248.598,1
2022-02-28,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.054721e+07,NaN,201.718,1
2022-01-31,0.00,0.00,0.000000e+00,0.000000e+00,-2.000,-2.000,1.000,9.582836e+06,NaN,185.931,1
2021-12-31,17480.75,7562.42,2.480148e+07,1.031895e+07,-0.042,-0.045,1.050,9.042219e+06,1418.79,178.505,1
...,...,...,...,...,...,...,...,...,...,...,...
2002-04-30,7399.49,3151.41,2.887130e+06,1.190754e+06,0.016,0.027,1.720,2.561574e+06,390.18,48.969,1
2002-03-31,7362.91,3123.33,2.828006e+06,1.158958e+06,-0.010,0.007,0.751,2.530613e+06,384.09,47.622,1
2002-02-28,7326.34,3095.25,2.841648e+06,1.150465e+06,-0.040,-0.039,0.983,2.532373e+06,387.87,47.786,1


In [25]:
df_ref  = pd.merge(df_ref, df1, how="left", left_index=True, right_index=True)
print(df_ref.shape)
df_ref

(245, 12)


,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,vegetable-oil_price,const,fred_price_idx
date,,,,,,,,,,,,
2022-04-30,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.227258e+07,NaN,NaN,1,NaN
2022-03-31,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.277204e+07,NaN,248.598,1,NaN
2022-02-28,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.054721e+07,NaN,201.718,1,674.724
2022-01-31,0.00,0.00,0.000000e+00,0.000000e+00,-2.000,-2.000,1.000,9.582836e+06,NaN,185.931,1,677.806
2021-12-31,17480.75,7562.42,2.480148e+07,1.031895e+07,-0.042,-0.045,1.050,9.042219e+06,1418.79,178.505,1,643.475
...,...,...,...,...,...,...,...,...,...,...,...,...
2002-04-30,7399.49,3151.41,2.887130e+06,1.190754e+06,0.016,0.027,1.720,2.561574e+06,390.18,48.969,1,128.000
2002-03-31,7362.91,3123.33,2.828006e+06,1.158958e+06,-0.010,0.007,0.751,2.530613e+06,384.09,47.622,1,126.100
2002-02-28,7326.34,3095.25,2.841648e+06,1.150465e+06,-0.040,-0.039,0.983,2.532373e+06,387.87,47.786,1,127.200


In [26]:
df_ref  = pd.merge(df_ref, df2, how="left", left_index=True, right_index=True)
print(df_ref.shape)
df_ref

(245, 13)


,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,vegetable-oil_price,const,fred_price_idx,bloomberg_price_index
date,,,,,,,,,,,,,
2022-04-30,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.227258e+07,NaN,NaN,1,NaN,1154.830000
2022-03-31,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.277204e+07,NaN,248.598,1,NaN,1160.192500
2022-02-28,0.00,0.00,0.000000e+00,0.000000e+00,NaN,NaN,0.000,1.054721e+07,NaN,201.718,1,674.724,883.657500
2022-01-31,0.00,0.00,0.000000e+00,0.000000e+00,-2.000,-2.000,1.000,9.582836e+06,NaN,185.931,1,677.806,886.600000
2021-12-31,17480.75,7562.42,2.480148e+07,1.031895e+07,-0.042,-0.045,1.050,9.042219e+06,1418.79,178.505,1,643.475,1080.743333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-04-30,7399.49,3151.41,2.887130e+06,1.190754e+06,0.016,0.027,1.720,2.561574e+06,390.18,48.969,1,128.000,98.092000
2002-03-31,7362.91,3123.33,2.828006e+06,1.158958e+06,-0.010,0.007,0.751,2.530613e+06,384.09,47.622,1,126.100,98.717500
2002-02-28,7326.34,3095.25,2.841648e+06,1.150465e+06,-0.040,-0.039,0.983,2.532373e+06,387.87,47.786,1,127.200,98.660000


In [39]:
df_ref.to_excel(r'E:\Projetos\vegetable_oil_mkt\mkt\__com_fertilizer.xlsx')



# 2SLS


In [27]:
# def compare_models_v2(dados, VI, explicada = "vegetable-oil_oecd_world-price",  explicativa = ["sunflower_exports"]):
  
#   dependent = dados[[explicada]]
#   exog = dados[['const'] + explicativa]
#   endog = None
#   instrument = None
#   res_ols  = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

#   dependent = dados[explicativa]
#   exog = dados[['const',VI]]
#   endog = None
#   instrument = None
#   res_first = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

#   # print(exog)
#   dados["z_hat"] = dados.sunflower_exports - res_first.resids
#   # dados[[explicativa, 'z_hat']].tail()
  
#   dependent = dados[[explicada]]
#   exog = dados[['const']]
#   endog = dados[explicativa]
#   instrument = dados[[VI]]
#   res_second = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

#   dependent = dados[[explicada]]
#   exog = dados[['const', 'z_hat']]
#   endog = None
#   instrument = None
#   res_direct = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

#   # print(linearmodels.iv.compare({"OLS": res_ols, "First": res_first , "2SLS": res_second, "Direct": res_direct}))
#   print(linearmodels.iv.compare({"OLS": res_ols, "2SLS": res_second, "Direct": res_direct}))

In [31]:
def compare_models(dados, VI, explicada, explicativa):
  
    dependent = dados[[explicada]]
    exog = dados[['const',explicativa]]
    endog = None
    instrument = None
    res_ols  = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

    dependent = dados[[explicativa]]
    exog = dados[['const',VI]]
    endog = None
    instrument = None
    res_first = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

    dados["z_hat"] = dados[explicada] - res_first.resids
    # dados[[explicativa, 'z_hat']].tail()

    dependent = dados[[explicada]]
    exog = dados[['const']]
    endog = dados[[explicativa]]
    instrument = dados[[VI]]
    res_second = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

    dependent = dados[[explicada]]
    exog = dados[['const', 'z_hat']]
    endog = None
    instrument = None
    res_direct = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument).fit(cov_type="robust")

    # print(linearmodels.iv.compare({"OLS": res_ols, "First": res_first , "2SLS": res_second, "Direct": res_direct}))
    print(linearmodels.iv.compare({"OLS": res_ols, "2SLS": res_second, "Direct": res_direct}))




In [32]:
df_ref.head()

,prod_total,exp_total,vol_prod_total,vol_expo_total,ΔP_total,ΔQ_total,elasticidade_total,ΣPnQo_total,price_total,vegetable-oil_price,const,fred_price_idx,bloomberg_price_index
date,,,,,,,,,,,,,
2022-04-30,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.227258e+07,NaN,NaN,1,NaN,1154.830000
2022-03-31,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.277204e+07,NaN,248.598,1,NaN,1160.192500
2022-02-28,0.00,0.00,0.00,0.00,NaN,NaN,0.00,1.054721e+07,NaN,201.718,1,674.724,883.657500
2022-01-31,0.00,0.00,0.00,0.00,-2.000,-2.000,1.00,9.582836e+06,NaN,185.931,1,677.806,886.600000
2021-12-31,17480.75,7562.42,24801477.59,10318952.58,-0.042,-0.045,1.05,9.042219e+06,1418.79,178.505,1,643.475,1080.743333


In [33]:
compare_models(dados=df_ref, VI='fred_price_idx', explicada='price_total', explicativa='vol_expo_total')

                           Model Comparison                          
                                 OLS             2SLS          Direct
---------------------------------------------------------------------
Dep. Variable            price_total      price_total     price_total
Estimator                        OLS          IV-2SLS             OLS
No. Observations                 241              241             241
Cov. Est.                     robust           robust          robust
R-squared                     0.7814           0.7677          0.3489
Adj. R-squared                0.7804           0.7668          0.3462
F-statistic                   1191.7           650.91          126.40
P-value (F-stat)              0.0000           0.0000          0.0000
==================     =============    =============   =============
const                         290.29           216.49          838.95
                            (24.460)         (9.4718)        (60.308)
vol_expo_total      

E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [35]:
compare_models(dados=df_ref, VI='bloomberg_price_index', explicada='price_total', explicativa='vol_expo_total')

                              Model Comparison                              
                                 OLS                    2SLS          Direct
----------------------------------------------------------------------------
Dep. Variable            price_total             price_total     price_total
Estimator                        OLS                 IV-2SLS             OLS
No. Observations                 241                     240             240
Cov. Est.                     robust                  robust          robust
R-squared                     0.7814                  0.7568          0.4038
Adj. R-squared                0.7804                  0.7558          0.4013
F-statistic                   1191.7                  472.46          120.36
P-value (F-stat)              0.0000                  0.0000          0.0000
==================     =============           =============   =============
const                         290.29                  196.68          830.89

E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [38]:
compare_models(dados=df_ref, VI='fred_price_idx', explicada='vol_expo_total', explicativa='price_total')

                               Model Comparison                              
                                    OLS               2SLS             Direct
-----------------------------------------------------------------------------
Dep. Variable            vol_expo_total     vol_expo_total     vol_expo_total
Estimator                           OLS            IV-2SLS                OLS
No. Observations                    241                241                241
Cov. Est.                        robust             robust             robust
R-squared                        0.7814             0.7680             1.0000
Adj. R-squared                   0.7804             0.7671             1.0000
F-statistic                      561.30             650.91          4.263e+10
P-value (F-stat)                 0.0000             0.0000             0.0000
==================     ================   ================   ================
const                         -8.89e+05         -1.626e+06      

E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [37]:
compare_models(dados=df_ref, VI='bloomberg_price_index', explicada='vol_expo_total', explicativa='price_total')

                                 Model Comparison                                 
                                    OLS                    2SLS             Direct
----------------------------------------------------------------------------------
Dep. Variable            vol_expo_total          vol_expo_total     vol_expo_total
Estimator                           OLS                 IV-2SLS                OLS
No. Observations                    241                     240                240
Cov. Est.                        robust                  robust             robust
R-squared                        0.7814                  0.7711             1.0000
Adj. R-squared                   0.7804                  0.7702             1.0000
F-statistic                      561.30                  472.46          3.744e+10
P-value (F-stat)                 0.0000                  0.0000             0.0000
==================     ================        ================   ================
cons

E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(
E:\Projetos\vegetable_oil_mkt\.env-win-tese\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [ ]:
ref_VI = df.columns.to_list()
ref_VI = [col for col in df.columns.to_list() if 'worldbank' in col]

for i in range(len(ref_VI)):
    dados = df.copy()
    print(dados.shape, f'VAR: {ref_VI[i]}', i)

    try:
    compare_models(dados, ref_VI[i], explicada = "vegetable-oil_oecd_world-price",  explicativa = "vegetable-oil_oecd_exports")
    except Exception as e:
    print(f'Error: {e} | {ref_VI[i]}')
    # print(i)

(33, 74) VAR: worldbank_agriculture-forestry-and-fishing-value-added-perc-of-gdp 0
                                                                             Model Comparison                                                                            
                                                              OLS                                                                 2SLS                             Direct
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                      vegetable-oil_oecd_world-price                                       vegetable-oil_oecd_world-price     vegetable-oil_oecd_world-price
Estimator                                                     OLS                                                              IV-2SLS                                OLS
No. Observations                                               33  

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                                                              Model Comparison                                                              
                                                              OLS                                    2SLS                             Direct
--------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                      vegetable-oil_oecd_world-price          vegetable-oil_oecd_world-price     vegetable-oil_oecd_world-price
Estimator                                                     OLS                                 IV-2SLS                                OLS
No. Observations                                               33                                      29                                 29
Cov. Est.                                                  robust                                  robust                             robust
R-squared    

In [ ]:
ref_VI = df.columns.to_list()
ref_VI = [col for col in df.columns.to_list() if 'worldbank' in col]

for i in range(len(ref_VI)):
  dados = df.copy().dropna()
  print(dados.shape, f'VAR: {ref_VI[i]}', i)

  try:
    compare_models(dados, ref_VI[i], explicada = "vegetable-oil_oecd_world-price",  explicativa = "sunflower_exports")
  except Exception as e:
    print(f'Error: {e} | {ref_VI[i]}')
  # print(i)

(10, 74) VAR: worldbank_agriculture-forestry-and-fishing-value-added-perc-of-gdp 0
                                                                        Model Comparison                                                                        
                                                     OLS                                                                 2SLS                             Direct
----------------------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable             vegetable-oil_oecd_world-price                                       vegetable-oil_oecd_world-price     vegetable-oil_oecd_world-price
Estimator                                            OLS                                                              IV-2SLS                                OLS
No. Observations                                      10                                                        

In [ ]:
m

# Estudo

In [ ]:
# Estimação do um efeito exogeno sobre um endogeno

dependent = dados[["vegetable-oil_oecd_world-price"]]
exog = dados[['const',"sunflower_exports"]]
endog = None
instrument = None

IV_model = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument)
IV_results = IV_model.fit(cov_type="robust")

print(IV_results)

                                  OLS Estimation Summary                                  
Dep. Variable:     vegetable-oil_oecd_world-price   R-squared:                      0.2008
Estimator:                                    OLS   Adj. R-squared:                 0.1742
No. Observations:                              32   F-statistic:                    15.095
Date:                            Wed, May 04 2022   P-value (F-stat)                0.0001
Time:                                    11:04:13   Distribution:                  chi2(1)
Cov. Estimator:                            robust                                         
                                                                                          
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const         

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [ ]:
dependent = dados[["vegetable-oil_oecd_world-price"]]
endog = dados[["sunflower_exports"]]
exog = dados[['sunflower_production']]
instrument = dados[['worldbank_gni-per-capita-atlas-method-current-us$']]

IV_model = linearmodels.IV2SLS(dependent = dependent, exog=exog, endog=endog, instruments=instrument)
IV_results = IV_model.fit(cov_type="robust")

print(IV_results)

                                IV-2SLS Estimation Summary                                
Dep. Variable:     vegetable-oil_oecd_world-price   R-squared:                      0.9247
Estimator:                                IV-2SLS   Adj. R-squared:                 0.9059
No. Observations:                              10   F-statistic:                    226.55
Date:                            Wed, May 04 2022   P-value (F-stat)                0.0000
Time:                                    10:28:20   Distribution:                  chi2(2)
Cov. Estimator:                            robust                                         
                                                                                          
                                  Parameter Estimates                                   
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
sunfl

In [ ]:
VI = ['worldbank_gdp-per-capita-ppp-current-international-$', 'worldbank_gross-capital-formation-perc-of-gdp']